In [ ]:
import glob
from IPython.display import HTML

from IPython.display import display, Image, clear_output, HTML
import time
import random
import pandas as pd

import ipywidgets as widgets
from jupyter_ui_poll import ui_events

import requests
from bs4 import BeautifulSoup
import json

# This function sends the data collected in a dictionary into a Google form via url
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}

    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]

    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

# Create a dictionary for all test images with image code (i1, i2... i64) as keys, and their path as values
image_dict = {}
for u in range(64):
    u += 1
    image_dict[f"i{u}"] = f"./ANS_Pics/ANS{u}.jpg"

# Create a list of all the image codes
# Copy the list and randomly shuffle the copy
image_list = []
for u in range(64):
    u += 1
    image_list.append(f"i{u}")

shuffled_image_list = image_list.copy()
random.shuffle(shuffled_image_list)

# Create a dictionary with the 4 dot ratios as keys, and the image codes that use each ratio as values
data_type = {"76" : ['i2', 'i6', 'i9', 'i10', 'i17', 'i20', 'i25', 'i29', 'i35', 'i39', 'i44', 'i46', 'i51', 'i53', 'i57', 'i60'],
             "43" : ['i3', 'i5', 'i8', 'i12', 'i14', 'i16', 'i19', 'i21', 'i23', 'i26', 'i28', 'i30', 'i34', 'i36', 'i40', 'i41', 'i47', 'i48', 'i49', 'i55', 'i56', 'i58', 'i61', 'i63'],
             "98" : ['i7', 'i13', 'i22', 'i32', 'i37', 'i43', 'i50', 'i64'],
             "109": ['i1', 'i4', 'i11', 'i15', 'i18', 'i24', 'i27', 'i31', 'i33', 'i38', 'i42', 'i45', 'i52', 'i54', 'i59', 'i62']
            }

# Create lists to classify the images into more dots on the right or left
Right = []
Left = []

for i in range(32):
    i += 1
    Right.append(f"i{i}")

for i in range(32):
    i += 33
    Left.append(f"i{i}")

# Create a dictionary to store the test info without using the global keyword
test_info = {
    'type': '',
    'choice': '',
    'time': '',
}

def ask_for_info():

    # Anonymized user ID, age, and gender are collected before the test and stored in variables
    instructions = widgets.HTML(value = """ <b> Please input your credentials below </b><br><br>

    / Guidance /<br>

    To generate an anonymous 4-letter unique user identifier please enter:<br>

    - 2 letters based on the initials (first and last name) of a childhood friend<br>

    - 2 letters based on the initials (first and last name) of a favourite actor / actress<br>

    <i> e.g. if your friend was called Peter Pan and film star was Brad Pitt<br>

    then your unique identifer would be PPBP. </i><br><br> """)

    user_id = widgets.Text(description = 'ID: ')
    user_age = widgets.IntText(description = 'Age: ')
    user_sex = widgets.ToggleButtons(options = ['F', 'M'], description = 'Sex: ', layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='auto', margin = '10px'))

    submit = widgets.Button(description = "Submit", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='auto', margin = '10px'))
    submit.style.button_color = '#5571AC'
    submit.on_click(register_event)

    panel = widgets.VBox([instructions, user_id, user_age, user_sex, submit], layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))

    box.children = [panel]
    wait_for_response()

    # Store user credentials inside the global dictionary
    if test_info["choice"] == "Submit":
        outcome["user_id"] = user_id.value
        outcome["age"] = user_age.value
        outcome["sex"] = user_sex.value
        box.children = []
        time.sleep(1)

    return outcome, box

def ask_for_consent():

    # Asks the user for consent to upload the test results
    data_consent = widgets.HTML(value = """<b>DATA CONSENT INFORMATION:</b><br>

    Please read:<br>

    we wish to record your response data<br>

    to an anonymised public data repository.<br>

    Your data will be used for educational teaching purposes<br>

    practising data analysis and visualisation.<br>

    <span style = 'color: #5571AC;'><b>Would you give us consent to record your test results?</b><br></span>""")

    choice = widgets.ToggleButtons(options = ['Yes', 'No'], layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='auto', margin = '10px'))

    submit = widgets.Button(description = "Submit", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='auto', margin = '10px'))
    submit.style.button_color = '#5571AC'
    submit.on_click(register_event)

    panel = widgets.HBox([data_consent, choice, submit], layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))

    box.children = [panel]
    wait_for_response()

    # Store the value of choice after submission
    if test_info["choice"] == "Submit":
        value.append(choice.value)
        box.children = []
        time.sleep(0.5)

    return test_info, value, box

def wait_for_response (timeout = -1, interval = 0.001, max_rate = 20, allow_interrupt = True):
    start = time.time()

    test_info['time'] = -1
    test_info['choice']= ""
    test_info['type'] = ""

    n_proc = int(max_rate*interval)+1

    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start + timeout):
                keep_looping = False

            # end loop if the event has occurred
            if allow_interrupt==True and test_info['choice']!="":
                keep_looping = False

            # add pause before looping to check events again
            time.sleep(interval)

    return test_info

# This function lets buttons register events when clicked
def register_event(btn):
    # display button description in output area
    test_info['type'] = 'click'
    test_info['choice'] = btn.description
    test_info['time'] = time.time()

    return test_info

# This is the main function for running the test
def run_test():

    # Creates a dictionary to store the number of correct and wrong responses
    # for each ratio (simplified by removing the :)
    Judge = {
            "C76":0,
            "C43":0,
            "C98":0,
            "C109":0,
            "W76":0,
            "W43":0,
            "W98":0,
            "W109":0
           }

    # Create a variable that stores the number of times the user didn't click in time
    no_click = 0

    # Create variables that collect the total number of correct and wrong responses
    # when the user clicks in time
    total_correct = 0
    total_wrong = 0

    # Create a variable to measure the cumulative time taken
    # for the user to respond within the 3s period.
    # n is the number of times the user responds,
    # which is collected to calculate the average response time.
    total_response_time = 0
    n = 0

    # Hide images and buttons when not needed
    box.children = [panel2]
    image.layout.display = 'none'
    panel1.layout.display = 'none'

    # Loops each image in the shuffled image list
    for i in shuffled_image_list:

        # Display the image through the corresponding path in the dictionary
        image.value = open(image_dict[i], 'rb').read()
        image.layout.display = ''

        # Remove the image from the screen after 0.75 seconds
        time.sleep(0.75)
        image.layout.display = 'none'

        # Left/right buttons appear
        # Collect the time at which the user is expected to start responding
        start = time.time()
        panel1.layout.display = ''

        # Accept user response until 3 seconds
        result = wait_for_response(timeout=3)

        choice = result['choice']
        if choice == "":
            panel1.layout.display = 'none'
            no_click += 1

        else:
            u = i

            # Check if the button clicked by the user matches the correct choice
            if choice == "Left":
                if u in Left:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"C{i}"]+= 1
                            total_correct += 1

                else:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"W{i}"]+= 1
                            total_wrong += 1

            elif choice == "Right":
                if u in Right:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"C{i}"]+= 1
                            total_correct += 1

                else:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"W{i}"]+= 1
                            total_wrong += 1

            # Once the user responds or 3 seconds is up, whichever is earlier,
            # the trial is complete
            # Buttons are removed before the onset of the intertrial buffer time
            panel1.layout.display = 'none'

            # Only when the user responds in time,
            # the response time is recorded and added to the total
            end = time.time()
            total_response_time += end - start
            n += 1

        # An intertrial of 1.5 seconds before the next trial
        time.sleep(1.5)

    box.children = []

    # Collect the time at which the entire test (64 trials) is complete
    # The total time taken to complete the test,
    # the average response time and the rate of correct answers are also calculated
    end_time = time.time()
    time_taken = end_time - start_time
    average = total_response_time/n
    correct_rate = total_correct/64

    # Assign the number of correct and wrong responses for each ratio to the corresponding variables
    C76 = Judge["C76"]
    C43 = Judge["C43"]
    C98 = Judge["C98"]
    C109 = Judge["C109"]
    W76 = Judge["W76"]
    W43 = Judge["W43"]
    W98 = Judge["W98"]
    W109 = Judge["W109"]

    # All relevant data collected from the test are stored in the outcome dictionary
    outcome["76 Correct"] = C76
    outcome["43 Correct"] = C43
    outcome["98 Correct"] = C98
    outcome["109 Correct"] = C109
    outcome["76 Wrong"] = W76
    outcome["43 Wrong"] = W43
    outcome["98 Wrong"] = W98
    outcome["109 Wrong"] = W109
    outcome["Not clicked in time"] = no_click
    outcome["Total Correct"] = total_correct
    outcome["Total Wrong"] = total_wrong
    outcome["Correct Rate"] = correct_rate
    outcome["Time Taken"] = time_taken
    outcome["Total response time"] = total_response_time
    outcome["Average response time"] = average

    time.sleep(0.5)

    return outcome

start = widgets.Button (description = "Start", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='auto', margin='10px'))
left = widgets.Button (description = "Left", layout = widgets.Layout(display='flex', flex_flow='column', width='250px', height='50px', align_items='center', margin='5px'))
right = widgets.Button (description = "Right", layout = widgets.Layout(display='flex', flex_flow='column', width='250px', height='50px', align_items='center', margin='5px'))
image = widgets.Image(layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))

start.style.button_color = '#5571AC'
start.on_click(register_event)
left.on_click(register_event)
right.on_click(register_event)

# box is the panel for the entire test
# Gives users a focused view by limiting it to a small-scaled border
box = widgets.VBox([], layout = widgets.Layout(width='500px', height='550px', border='2px solid'))
display(box)

outcome = {}

ask_for_info()

# This part tells the user what to expect in the test to increase the representativeness of the data collected
start_guide = widgets.HTML("""<b>This is an approximate number system (ANS) test. </b><br><br>
    / Instructions /<br>
    For each trial, a test image will be shown for <b>0.75 seconds</b>. <br>
    2 buttons will appear on the top, displaying "Left" and "Right". <br>
    You need to choose the side with <b>more dots</b> <br>
    and click on the corresponding button <b>as fast as you can</b>. <br><br>
    <span style = "color: #5571AC;"><b>Click the start button when you are ready to go.</b></span>""")

panel = widgets.VBox([start_guide, start], layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))
panel1 = widgets.HBox([left, right], layout = widgets.Layout(display='flex', flex_flow='row', align_items='center', width='500px'))
panel2 = widgets.VBox([panel1, image], layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))

box.children = [panel]

# The test is started upon clicking the start button and the time is recorded
result = wait_for_response()
box.children = []
start_time = time.time()

run_test()

# Use a list to contain the choice of consent without using the global keyword
value = []
ask_for_consent()

# Display test result that users might be interested in
result = widgets.HTML(value = f"""Thanks for taking the test, user <span style = 'color: #5571AC;'>{outcome['user_id']}</span> :D<br>
You took <b>{round(outcome["Time Taken"]/60,2)} minutes</b> to complete the test.<br>
The average response time is <b>{round(outcome["Average response time"], 2)} seconds</b>.<br>
Your score is <b>{outcome["Total Correct"]} out of 64</b>.<br>""", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))

choice = test_info["choice"]
if "Yes" in value:

    # Send the data collected in the outcome to the Google form upon consent
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfybwZCx7dJd8O9T_gNb3Am11ghlJK77FL3OEGrX93vE9JQ2Q/viewform?usp=sf_link"
    send_to_google_form(outcome, form_url)

    text = widgets.HTML(value = """<br>Thanks for your participation.<br>

    Your data has been uploaded.<br>

    Please contact philip.lewis@ucl.ac.uk<br>

    if you have any questions or concerns<br>

    regarding the stored results.""")

    box.children = [result, text]

elif "No" in value:

    # Data not uploaded if no consent is given
    text = widgets.HTML(value = "<br>No problem, we hope you enjoyed the test!", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))
    box.children = [result, text]

else:

    # Debugging
    raise Exception("Incorrect value")
